In [28]:
from thainlplib import ThaiWordSegmentLabeller
import numpy as np
import tensorflow as tf
    
def word_segmenter(sentence):
    # Pretrained model weights location
    saved_model_path='saved_model'

    # Input text
    #thai_txt = open("/Users/korn/Desktop/thai-word-data.txt", "r")
    #thai_txt = sentence.read()
    thai_txt = sentence
    text = thai_txt

    # Convert text to labels
    inputs = [ThaiWordSegmentLabeller.get_input_labels(text)]
    lengths = [len(text)]

    def nonzero(a):
        return [i for i, e in enumerate(a) if e != 0]

    def split(s, indices):
        return [s[i:j] for i,j in zip(indices, indices[1:]+[None])]

    with tf.Session() as session:
        # Read model weights
        model = tf.saved_model.loader.load(session, [tf.saved_model.tag_constants.SERVING], saved_model_path)

        # Get model input variables
        graph = tf.get_default_graph()
        g_inputs = graph.get_tensor_by_name('IteratorGetNext:1')
        g_lengths = graph.get_tensor_by_name('IteratorGetNext:0')
        g_training = graph.get_tensor_by_name('Placeholder_1:0')
        g_outputs = graph.get_tensor_by_name('boolean_mask_1/Gather:0')

        # Segment the text
        y = session.run(g_outputs, feed_dict = {g_inputs: inputs, g_lengths: lengths, g_training: False})

        # Mark word boundaries with pipe character
        #for w in split(text, nonzero(y)): print(w, end='|')
        #print()
        
        """
        print('Word')
        for w in split(text, nonzero(y)):
            print('{}'.format(w))
        """

        
        result = []
        for w in split(text, nonzero(y)):
            result.append(w)
            
        return result
        

In [41]:
word = word_segmenter("ถ้าเราเชื่อว่า หุ้นคือสินทรัพย์ เราจะทำการสะสมหุ้นไปเรื่อยๆ ซื้อหุ้น เวลาที่หุ้นราคาถูก เวลามีข่าวร้าย")
print(word)

INFO:tensorflow:Restoring parameters from b'saved_model/variables/variables'
['ถ้า', 'เรา', 'เชื่อ', 'ว่า', ' ', 'หุ้น', 'คือ', 'สินทรัพย์', ' ', 'เรา', 'จะ', 'ทำ', 'การ', 'สะสม', 'หุ้น', 'ไป', 'เรื่อย', 'ๆ', ' ', 'ซื้อ', 'หุ้น', ' ', 'เวลา', 'ที่', 'หุ้น', 'ราคา', 'ถูก', ' ', 'เวลา', 'มี', 'ข่าว', 'ร้าย']


In [42]:
from pythainlp.tag import pos_tag
pos_tag = pos_tag(word,engine='old')
print(pos_tag)

[('ถ้า', 'JSBR'), ('เรา', 'PPRS'), ('เชื่อ', 'VACT'), ('ว่า', 'JSBR'), (' ', None), ('หุ้น', 'NCMN'), ('คือ', 'VSTA'), ('สินทรัพย์', None), (' ', None), ('เรา', 'PPRS'), ('จะ', 'XVBM'), ('ทำ', 'VACT'), ('การ', 'FIXN'), ('สะสม', 'VSTA'), ('หุ้น', 'NCMN'), ('ไป', 'XVAE'), ('เรื่อย', 'ADVN'), ('ๆ', 'NCMN'), (' ', None), ('ซื้อ', 'VACT'), ('หุ้น', 'NCMN'), (' ', None), ('เวลา', 'NCMN'), ('ที่', 'PREL'), ('หุ้น', 'NCMN'), ('ราคา', 'NCMN'), ('ถูก', 'XVAM'), (' ', None), ('เวลา', 'NCMN'), ('มี', 'VSTA'), ('ข่าว', 'NCMN'), ('ร้าย', 'VATT')]


In [43]:
from pythainlp.rank import rank
freq = rank(word)
print(freq)

Counter({' ': 5, 'หุ้น': 4, 'เรา': 2, 'เวลา': 2, 'ถ้า': 1, 'เชื่อ': 1, 'ว่า': 1, 'คือ': 1, 'สินทรัพย์': 1, 'จะ': 1, 'ทำ': 1, 'การ': 1, 'สะสม': 1, 'ไป': 1, 'เรื่อย': 1, 'ๆ': 1, 'ซื้อ': 1, 'ที่': 1, 'ราคา': 1, 'ถูก': 1, 'มี': 1, 'ข่าว': 1, 'ร้าย': 1})


In [44]:
print(len(word))
print(len(pos_tag))
print(len(freq))

32
32
23


In [49]:
result = []
for w, p in zip(word, pos_tag):
    result.append({
        'word' : w,
        'pos_tag' : p
    })

In [46]:
print(result)

[{'word': 'ถ้า', 'pos_tag': ('ถ้า', 'JSBR')}, {'word': 'เรา', 'pos_tag': ('เรา', 'PPRS')}, {'word': 'เชื่อ', 'pos_tag': ('เชื่อ', 'VACT')}, {'word': 'ว่า', 'pos_tag': ('ว่า', 'JSBR')}, {'word': ' ', 'pos_tag': (' ', None)}, {'word': 'หุ้น', 'pos_tag': ('หุ้น', 'NCMN')}, {'word': 'คือ', 'pos_tag': ('คือ', 'VSTA')}, {'word': 'สินทรัพย์', 'pos_tag': ('สินทรัพย์', None)}, {'word': ' ', 'pos_tag': (' ', None)}, {'word': 'เรา', 'pos_tag': ('เรา', 'PPRS')}, {'word': 'จะ', 'pos_tag': ('จะ', 'XVBM')}, {'word': 'ทำ', 'pos_tag': ('ทำ', 'VACT')}, {'word': 'การ', 'pos_tag': ('การ', 'FIXN')}, {'word': 'สะสม', 'pos_tag': ('สะสม', 'VSTA')}, {'word': 'หุ้น', 'pos_tag': ('หุ้น', 'NCMN')}, {'word': 'ไป', 'pos_tag': ('ไป', 'XVAE')}, {'word': 'เรื่อย', 'pos_tag': ('เรื่อย', 'ADVN')}, {'word': 'ๆ', 'pos_tag': ('ๆ', 'NCMN')}, {'word': ' ', 'pos_tag': (' ', None)}, {'word': 'ซื้อ', 'pos_tag': ('ซื้อ', 'VACT')}, {'word': 'หุ้น', 'pos_tag': ('หุ้น', 'NCMN')}, {'word': ' ', 'pos_tag': (' ', None)}, {'word': 'เวล

In [62]:
result = []
for i in range(len(word)):
    result.append({
        'word' : word[i],
        'pos_tag' : pos_tag[i][1],
        'freq' : freq[word[i]]
    })

In [63]:
print(result)

[{'word': 'ถ้า', 'pos_tag': 'JSBR', 'freq': 1}, {'word': 'เรา', 'pos_tag': 'PPRS', 'freq': 2}, {'word': 'เชื่อ', 'pos_tag': 'VACT', 'freq': 1}, {'word': 'ว่า', 'pos_tag': 'JSBR', 'freq': 1}, {'word': ' ', 'pos_tag': None, 'freq': 5}, {'word': 'หุ้น', 'pos_tag': 'NCMN', 'freq': 4}, {'word': 'คือ', 'pos_tag': 'VSTA', 'freq': 1}, {'word': 'สินทรัพย์', 'pos_tag': None, 'freq': 1}, {'word': ' ', 'pos_tag': None, 'freq': 5}, {'word': 'เรา', 'pos_tag': 'PPRS', 'freq': 2}, {'word': 'จะ', 'pos_tag': 'XVBM', 'freq': 1}, {'word': 'ทำ', 'pos_tag': 'VACT', 'freq': 1}, {'word': 'การ', 'pos_tag': 'FIXN', 'freq': 1}, {'word': 'สะสม', 'pos_tag': 'VSTA', 'freq': 1}, {'word': 'หุ้น', 'pos_tag': 'NCMN', 'freq': 4}, {'word': 'ไป', 'pos_tag': 'XVAE', 'freq': 1}, {'word': 'เรื่อย', 'pos_tag': 'ADVN', 'freq': 1}, {'word': 'ๆ', 'pos_tag': 'NCMN', 'freq': 1}, {'word': ' ', 'pos_tag': None, 'freq': 5}, {'word': 'ซื้อ', 'pos_tag': 'VACT', 'freq': 1}, {'word': 'หุ้น', 'pos_tag': 'NCMN', 'freq': 4}, {'word': ' ', '

In [68]:
import pandas as pd
pd.DataFrame(result)


,freq,pos_tag,word
0,1,JSBR,ถ้า
1,2,PPRS,เรา
2,1,VACT,เชื่อ
3,1,JSBR,ว่า
4,5,None,
5,4,NCMN,หุ้น
6,1,VSTA,คือ
7,1,None,สินทรัพย์
8,5,None,
9,2,PPRS,เรา


In [69]:
from pythainlp.sentiment import sentiment
sentiment("ถ้าเราเชื่อว่า หุ้นคือสินทรัพย์ เราจะทำการสะสมหุ้นไปเรื่อยๆ ซื้อหุ้น เวลาที่หุ้นราคาถูก เวลามีข่าวร้าย")

'neg'